In [1]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483",
    security_token="nc20DsgxOHWVR9OHZ78s7TRp",
)

In [2]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,Vigencia__c, Corredor__c,Pa_s_del_Paciente__c, Habilitado__c from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "Vigencia__c",
        "Corredor__c",
        "Pa_s_del_Paciente__c",
        "Habilitado__c",
    ]
]
Empresas_SF.shape

(7008, 11)

In [3]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(6715, 5)

In [4]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas.shape

(7014, 16)

In [5]:
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Empleador__c": "Empleador",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",        
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
        "Vigencia__c": "Vigente",
        "Corredor__c": "Corredor",
        "Pa_s_del_Paciente__c": "Pais",
        "Habilitado__c": "Habilitado",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Empleador",
        "Holding",
        "Campaña",        
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
        "Vigente",
        "Corredor",
        "Pais",
        "Habilitado",
    ]
]

In [6]:

Programas_empresas.to_excel("C:/Users/jmartinez/Documents/Empresas2.xlsx", index=False)
#.to_excel("F:/Empresas SF/reportes/empresas2.xlsx", index=False)
Programas_empresas.head()

,Id Account Empresa,Nombre Empresa,Empleador,Holding,Campaña,Poliza,Programas_por_Empresas__c,Codigo EMP,Codigo A,Vigente,Corredor,Pais,Habilitado
0,0018c00002f27paAAA,Forus - Dado De Baja - Sin Servicio,Forus - Dado De Baja - Sin Servicio,Sin Holding Asociado,Forus,Sin Póliza Asociada,a1W8c0000083oJOEAY,EMP9004,A-8034,NO,None,a1U8c000005dAtnEAE,False
1,0018c00002ghxCCAAY,BICE Vida S.A.,BICE Vida S.A.,Sin Holding Asociado,BICE Vida,Sin Póliza Asociada,a1W8c0000083qXGEAY,EMP9090,A-8117,None,None,a1U8c000005dAtnEAE,True
2,0018c00002cY37kAAC,Mercado Libre Ltda.,Mercado Libre Ltda.,Mercado Libre,Sura,7720,a1W8c0000083jGVEAY,EMP5617,A-2356,SI,None,a1U8c000005dAtnEAE,True
3,0018c00002f2W7UAAU,Minera Escondida Ltda.,Minera Escondida Ltda.,Grupo BHP,Vida Security,284247,a1W8c0000083rZnEAI,EMP9011,A-9239,SI,None,a1U8c000005dAtnEAE,True
4,0018c00002f2WJyAAM,Minera Escondida Ltda.,Minera Escondida Ltda.,Grupo BHP,Vida Security,282350,a1W8c0000083rZoEAI,EMP9013,A-9240,SI,None,a1U8c000005dAtnEAE,True


Finalizacion de Recorrido por las empresas


Ahora comienza la recoleccion de las fichas y sus datos

In [4]:
# Citas de Servicio
results = sf.query_all(
    """
    SELECT Id, clinica__c,AccountId,CreatedDate,RUT__c,Work_TypeGroup__c,Status,Interacci_n__c,AppointmentNumber,Tipo_de_la_cita__c,Empresa__c from ServiceAppointment
    """
)
Citas_de_servicio = pd.DataFrame(results["records"])
Citas_de_servicio['Empresas'] = np.nan
#Citas_de_servicio['RUT__c'].map(pacientessf_empresas.set_index('IdentificationId__pc_x',verify_integrity=False)['Name']) 
Citas_de_servicio['Empresas Id'] = np.nan
Citas_de_servicio = Citas_de_servicio[
    [
        "AppointmentNumber",        
        "RUT__c",  
        'Empresas',      
        "CreatedDate",
        "Tipo_de_la_cita__c",
        'Empresas Id',        
        "AccountId",
        'Clinica__c',
        'Empresa__c'
        
    ]
]
Citas_de_servicio["AppointmentNumber"] = 'F' + Citas_de_servicio["AppointmentNumber"]
    
Citas_de_servicio.head()


,AppointmentNumber,RUT__c,Empresas,CreatedDate,Tipo_de_la_cita__c,Empresas Id,AccountId,Clinica__c
0,FSA-16763,128060006,NaN,2023-04-20T23:17:29.000+0000,Video,NaN,0018c00002gl5fhAAA,Nutrición
1,FSA-17166,186247981,NaN,2023-04-24T22:22:01.000+0000,Video,NaN,0018c00002iwWAuAAM,Nutrición
2,FSA-17173,186318072,NaN,2023-04-24T22:56:10.000+0000,Phone,NaN,0018c00002azrebAAA,Nutrición
3,FSA-17196,97073430,NaN,2023-04-25T14:21:40.000+0000,Phone,NaN,0018c00002amLcMAAU,Nutrición
4,FSA-16876,183801422,NaN,2023-04-21T19:42:01.000+0000,Phone,NaN,0018c00002amLvkAAE,Nutrición


In [8]:
# Citas de Servicio Orientacion
results = sf.query_all(
    """
    SELECT Id, AccountId,CreatedDate,RUT__c,WorkTypeId,Work_TypeGroup__c,Status,Interacci_n__c,Tipo_de_la_cita__c,AppointmentNumber,Empresa__c from ServiceAppointment
    """
)
Citas_de_servicio2 = pd.DataFrame(results["records"])
Citas_de_servicio2["Clinica__c"] = "Orientación Médica"
Citas_de_servicio2["Tipo_de_la_cita__c"] = "Agendamiento"
#Citas_de_servicio2["Empresas"] = np.nan
Citas_de_servicio2["Empresas Id"] = np.nan
Citas_de_servicio2 = Citas_de_servicio2[
    [
        "AppointmentNumber",
        "RUT__c",
        "Empresas",
        "CreatedDate",
        "Tipo_de_la_cita__c",
        "Empresas Id",
        "AccountId",
        "Clinica__c",
        'Empresa__c'
    ]
]
Citas_de_servicio2.head(10)

,AppointmentNumber,RUT__c,Empresas,CreatedDate,Tipo_de_la_cita__c,Empresas Id,AccountId,Clinica__c
0,SA-16763,128060006,NaN,2023-04-20T23:17:29.000+0000,Agendamiento,NaN,0018c00002gl5fhAAA,Orientación Médica
1,SA-17166,186247981,NaN,2023-04-24T22:22:01.000+0000,Agendamiento,NaN,0018c00002iwWAuAAM,Orientación Médica
2,SA-17173,186318072,NaN,2023-04-24T22:56:10.000+0000,Agendamiento,NaN,0018c00002azrebAAA,Orientación Médica
3,SA-17196,97073430,NaN,2023-04-25T14:21:40.000+0000,Agendamiento,NaN,0018c00002amLcMAAU,Orientación Médica
4,SA-16876,183801422,NaN,2023-04-21T19:42:01.000+0000,Agendamiento,NaN,0018c00002amLvkAAE,Orientación Médica
5,SA-16938,185378438,NaN,2023-04-21T23:14:56.000+0000,Agendamiento,NaN,0018c00002b0I14AAE,Orientación Médica
6,SA-18511,97711593,NaN,2023-05-08T18:38:54.000+0000,Agendamiento,NaN,0018c00002ayFXJAA2,Orientación Médica
7,SA-17080,144767373,NaN,2023-04-24T17:29:43.000+0000,Agendamiento,NaN,0018c00002amV7EAAU,Orientación Médica
8,SA-17093,13727468k,NaN,2023-04-24T18:29:56.000+0000,Agendamiento,NaN,0018c00002f2THIAA2,Orientación Médica
9,SA-18572,192012783,NaN,2023-05-08T21:34:25.000+0000,Agendamiento,NaN,0018c00002axwg7AAA,Orientación Médica


In [6]:
#unir dos citas
Gran_Citas_deservicio = pd.concat([Citas_de_servicio, Citas_de_servicio2], axis=0)
Gran_Citas_deservicio = Gran_Citas_deservicio.rename(
    columns={
        "AppointmentNumber": "Codigo Ficha",
        "RUT__c": "Rut",
        "Empresas": "Empleador",
        "CreatedDate": "Fecha",
        "Tipo_de_la_cita__c": "Canal",
        "Empresas Id": "Cod. Empresa",
        "AccountId": "Id Paciente",        
        'Clinica__c':'Clínica'
        'Empresa__c'
    }
)

Gran_Citas_deservicio.shape

(34118, 8)

In [9]:
#Clinica_deporte
results = sf.query_all(
    """
   SELECT id,Name,Rut__c,Nombre_Empresa__c,CreatedDate,Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Deporte__c
     """
)
ficha_deporte = pd.DataFrame(results["records"])

ficha_deporte = ficha_deporte[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_deporte['clinica'] = 'Deporte'
ficha_deporte = ficha_deporte.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",        
        'clinica':'Clínica'
    }
)
ficha_deporte.shape


(2507, 8)

In [10]:
ficha_deporte

,Codigo Ficha,Rut,Empleador,Fecha,Canal,Cod. Empresa,Id Paciente,Clínica
0,FCD-0022,266339437,J.E.J Ingeniería S.A.,2022-11-23T18:49:46.000+0000,None,a1Y8c00000CmDMoEAN,0018c00002bHWekAAG,Deporte
1,FCD-0023,185852482,MetLife Chile Seguros De Vida S.A.,2022-11-23T18:57:36.000+0000,None,a1Y8c00000CkuD6EAJ,0018c00002axZ3AAAU,Deporte
2,FCD-0024,98222456,C.C.A.F. De Los Andes,2022-11-23T19:02:22.000+0000,None,a1Y8c00000C5auFEAR,0018c00002amLUqAAM,Deporte
3,FCD-0025,182926353,Empresa De Servicios Externos ACHS S.A.,2022-11-23T19:12:18.000+0000,None,a1Y8c00000CkxsZEAR,0018c00002ay3NxAAI,Deporte
4,FCD-0027,132714541,Colaboradores Care Assistance Chile SPA,2022-11-23T20:24:24.000+0000,None,a1Y8c00000C6VvQEAV,0018c00002amLV2AAM,Deporte
...,...,...,...,...,...,...,...,...
2502,FCD-1823,VIGA930419MDFLML09,OMG MX,2023-03-29T20:37:17.000+0000,Videollamada,a1Y8c00000DyOPvEAN,0018c00002gimp3AAA,Deporte
2503,FCD-1824,186247981,Servicios Informáticos Integrales Full,2023-03-29T20:43:25.000+0000,Llamada,a1Y8c00000DzRjsEAF,0018c00002iwWAuAAM,Deporte
2504,FCD-1825,184618184,Clínica Alemana,2023-03-29T20:52:58.000+0000,Llamada,a1Y8c00000C5e40EAB,0018c00002ao5aVAAQ,Deporte
2505,FCD-1826,193081681,Clínica Alemana,2023-03-29T21:54:40.000+0000,Llamada,a1Y8c00000DyIsAEAV,0018c00002b02drAAA,Deporte
